## imports

In [11]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StructField, StringType, DateType, IntegerType
from pyspark.sql.functions import col, explode, array, struct, expr, sum, lit

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## define bucket

In [2]:
input_bucket = 's3://dh-ud-dend-capstone/stage0_raw/google'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Google open covid data

In [3]:
def read_dataset(dataset):
    return spark.read.format('csv').options(header='true').load(input_bucket + dataset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Index file

In [4]:
#FIXME2: change s3 to s3a

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_index = read_dataset('/index.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#we only need the country_code so far
df_index = df_index.select('key', 'country_code')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
#FIXME: save to stage1 also

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Epidemiology dataset

In [11]:
schema = StructType([ \
    StructField("date",DateType(),True), \
    StructField("key",StringType(),True), \
    StructField("new_confirmed",IntegerType(),True), \
    StructField("new_deceased", IntegerType(), True), \
    StructField("new_recovered", IntegerType(), True), \
    StructField("new_tested", IntegerType(), True), \
    StructField("total_confirmed",IntegerType(),True), \
    StructField("total_deceased", IntegerType(), True), \
    StructField("total_recovered", IntegerType(), True), \
    StructField("total_tested", IntegerType(), True), \
  ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#df_epi = read_dataset('/epidemiology.csv')
df_epi = spark.read.format('csv').options(header='true').schema(schema).load(input_bucket + '/epidemiology.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_epi = df_epi.withColumn("month",F.month("date")).withColumn("year",F.year("date"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_epi_with_index = df_epi.join(df_index, on=['key'], how='inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_epi_with_index.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3930988

In [16]:
df_epi_with_index.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- key: string (nullable = true)
 |-- date: date (nullable = true)
 |-- new_confirmed: integer (nullable = true)
 |-- new_deceased: integer (nullable = true)
 |-- new_recovered: integer (nullable = true)
 |-- new_tested: integer (nullable = true)
 |-- total_confirmed: integer (nullable = true)
 |-- total_deceased: integer (nullable = true)
 |-- total_recovered: integer (nullable = true)
 |-- total_tested: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- country_code: string (nullable = true)

In [17]:
# partion by country, year, month
df_epi_with_index.write.mode('overwrite').partitionBy("country_code").parquet("s3://dh-ud-dend-capstone/stage1_prepared_and_partitioned/google/epidemiology.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Validation

In [19]:
#reload dataset
df_epi_pq = spark.read.parquet("s3://dh-ud-dend-capstone/stage1_prepared_and_partitioned/google/epidemiology.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
assert df_epi_with_index.count() == df_epi_pq.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df_epi_pq.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- key: string (nullable = true)
 |-- date: date (nullable = true)
 |-- new_confirmed: integer (nullable = true)
 |-- new_deceased: integer (nullable = true)
 |-- new_recovered: integer (nullable = true)
 |-- new_tested: integer (nullable = true)
 |-- total_confirmed: integer (nullable = true)
 |-- total_deceased: integer (nullable = true)
 |-- total_recovered: integer (nullable = true)
 |-- total_tested: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- country_code: string (nullable = true)

### TODO Findings
* convert date to date
* convert metrics to int
* date is not continous => check
* null values in metrics

## Mobility dataset

### Notes
* no transpose because we want to integrate it in big fact table
* and: not too many mobility types

In [31]:
schema_mobility = StructType([ \
    StructField("date",DateType(),True), \
    StructField("key",StringType(),True), \
    StructField("mobility_retail_and_recreation",IntegerType(),True), \
    StructField("mobility_grocery_and_pharmacy", IntegerType(), True), \
    StructField("mobility_parks", IntegerType(), True), \
    StructField("mobility_transit_stations", IntegerType(), True), \
    StructField("mobility_workplaces",IntegerType(),True), \
    StructField("mobility_residential", IntegerType(), True), \
  ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df_mob = spark.read.format('csv').options(header='true').schema(schema_mobility).load(input_bucket + '/mobility.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# show some records
df_mob.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---+------------------------------+-----------------------------+--------------+-------------------------+-------------------+--------------------+
|      date|key|mobility_retail_and_recreation|mobility_grocery_and_pharmacy|mobility_parks|mobility_transit_stations|mobility_workplaces|mobility_residential|
+----------+---+------------------------------+-----------------------------+--------------+-------------------------+-------------------+--------------------+
|2020-02-15| AE|                             0|                            4|             5|                        0|                  2|                   1|
|2020-02-15| AF|                            -9|                          -10|             0|                       -2|                -28|                   5|
|2020-02-15| AG|                            -6|                           -1|            -6|                       -1|                  0|                   3|
|2020-02-15| AO|                        

In [34]:
df_mob.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date: date (nullable = true)
 |-- key: string (nullable = true)
 |-- mobility_retail_and_recreation: integer (nullable = true)
 |-- mobility_grocery_and_pharmacy: integer (nullable = true)
 |-- mobility_parks: integer (nullable = true)
 |-- mobility_transit_stations: integer (nullable = true)
 |-- mobility_workplaces: integer (nullable = true)
 |-- mobility_residential: integer (nullable = true)

In [35]:
df_mob.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2052120

In [36]:
df_mob = df_mob.withColumn("month",F.month("date")).withColumn("year",F.year("date"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
df_mob_with_index = df_mob.join(df_index, on=['key'], how='inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df_mob_with_index.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2052120

In [40]:
# partion by country
df_mob_with_index.write.mode('overwrite').partitionBy("country_code").parquet("s3://dh-ud-dend-capstone/stage1_prepared_and_partitioned/google/mobility.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### TODO Findings
* convert date to date
* convert metrics to int or double?
* date is not continous => check
* null values in metrics
* transpose metrics?
* read with schema and do the summary again

## Search Trends dataset

In [7]:
gl_st_file = '/google-search-trends.csv'
df_st = spark.read.format('csv').options(header='true').load(input_bucket + gl_st_file)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
def transpose_search_trends(df, by):
    #source: https://stackoverflow.com/questions/55378047/pyspark-dataframe-melt-columns-into-rows
    cols, dtypes = zip(*((c, t) for (c, t) in df.dtypes if c not in by))
    kvs = explode(array([
      struct(lit(c).alias("symptom"), col(c).alias("search_trend")) for c in cols
    ])).alias("kvs")
    return df.select(by + [kvs]).select(by + ["kvs.symptom", "kvs.search_trend"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_st_transposed = transpose_search_trends(df_st, ['date', 'key'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
assert ((len(df_st.columns)-2) * df_st.count()) == df_st_transposed.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_st_transposed.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date: string (nullable = true)
 |-- key: string (nullable = true)
 |-- symptom: string (nullable = false)
 |-- search_trend: string (nullable = true)

In [14]:
df_st_transposed.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---+--------------------+------------+
|      date|key|             symptom|search_trend|
+----------+---+--------------------+------------+
|2020-01-01| AU|search_trends_abd...|        3.56|
|2020-01-01| AU|search_trends_abd...|        5.38|
|2020-01-01| AU|  search_trends_acne|       10.76|
|2020-01-01| AU|search_trends_act...|        0.52|
|2020-01-01| AU|search_trends_acu...|        0.35|
+----------+---+--------------------+------------+
only showing top 5 rows

In [15]:
#df_st_transposed.summary().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# FIXME: filter null values

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# replace search_trend in symptom
df_st_transposed = df_st_transposed.withColumn('symptom',expr('substring(symptom, 15)'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# convert date to date
df_st_transposed = df_st_transposed.withColumn("date", df_st_transposed["date"].cast("date"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# convert search_trend to double
df_st_transposed = df_st_transposed.withColumn("search_trend", df_st_transposed["search_trend"].cast("double"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# join with index
df_st_transposed = df_st_transposed.join(df_index, on=['key'], how='inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# partion by country
df_st_transposed.write.mode('overwrite').partitionBy("country_code").parquet("s3://dh-ud-dend-capstone/stage1_prepared_and_partitioned/google/searchtrends.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### TODO Findings
* convert date to date
* convert metrics to double
* date is not continous => check
* null values in metrics
* transpose metrics
* read with schema and do the summary again